In [1]:
from torchvision import utils
from dataloader_augmented import *
from utils import *
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import torch
import time
from basic_fcn import FCN
from datetime import datetime

In [2]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [3]:
# Change name to FCN to use this model instead I think

class FCN_bak(torch.nn.Module):

    def __init__(self, n_class):
        super(FCN_bak, self).__init__()
        self.n_class = n_class
        self.conv1   = nn.Conv2d(3, 32, kernel_size=(3,5), stride=(2,4), padding=1, dilation=1)
        self.bnd1    = nn.BatchNorm2d(32)
        self.conv2   = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd2    = nn.BatchNorm2d(64)
        self.conv3   = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd3    = nn.BatchNorm2d(128)
        self.conv4   = nn.Conv2d(128,256, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd4    = nn.BatchNorm2d(256)
        self.conv5   = nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1, dilation=1)
        self.bnd5    = nn.BatchNorm2d(512)
        self.relu    = nn.ReLU(inplace=True)
        
        self.deconv1 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn1     = nn.BatchNorm2d(256)
        self.deconv2 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn2     = nn.BatchNorm2d(128)
        self.deconv3 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn3     = nn.BatchNorm2d(64)
        self.deconv4 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.bn4     = nn.BatchNorm2d(32)
        self.deconv5  = nn.ConvTranspose2d(32, 3, kernel_size=(3, 5), stride=(2,4), padding=1, dilation=1, output_padding=1)
        self.bn5= nn.BatchNorm2d(3)
        self.classifier = nn.Conv2d(3,n_class, kernel_size=1, stride=1, padding=0, dilation=1)
        
    def forward(self, x):
        pool = nn.MaxPool2d(2, stride=2,return_indices = True)
        unpool = nn.MaxUnpool2d(2, stride=2)
        
        x1, indice1 = pool(self.relu(self.conv1(x)))
        x2, indice2 = pool(self.relu(self.conv2(self.bnd1(x1))))
        x3, indice3 = pool(self.relu(self.conv3(self.bnd2(x2))))
        x4, indice4 = pool(self.relu(self.conv4(self.bnd3(x3))))
        x5, indice5 = pool(self.relu(self.conv5(self.bnd4(x4))))
        
        z1 = self.deconv1(self.bnd5(self.relu(unpool((x5), indice5))))
        z2 = self.deconv2(self.bn1(self.relu(unpool((z1), indice4))))
        z3 = self.deconv3(self.bn2(self.relu(unpool((z2), indice3))))
        z4 = self.deconv4(self.bn3(self.relu(unpool((z3), indice2))))
        z5 = self.deconv5(self.bn4(self.relu(unpool((z4), indice1))))
        
        out_decoder = self.classifier(self.bn5(z5))                  

        return out_decoder  # size=(N, n_class, x.H/1, x.W/1)

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
def print_GPU_stats():
    print("total GPU Mem: ", torch.cuda.get_device_properties(device).total_memory)
    print("total GPU Cached: ", torch.cuda.memory_cached(device))
    print("total GPU Allocated: ", torch.cuda.memory_allocated(device))
    print("Available GB: ", (torch.cuda.get_device_properties(device).total_memory - torch.cuda.memory_allocated(device))/(10**9))
print_GPU_stats()

cuda:0
total GPU Mem:  11721506816
total GPU Cached:  0
total GPU Allocated:  0
Available GB:  11.721506816


In [5]:
batch_size = 5
train_dataset = AugmentedCityScapesDataset(csv_file='train_part.csv')
# train_dataset = CityScapesDataset(csv_file='train_part.csv')
val_dataset = CityScapesDataset(csv_file='val_part.csv')
test_dataset = CityScapesDataset(csv_file='test_part.csv')
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          num_workers=2,
                          shuffle=True)
val_loader = DataLoader(dataset=val_dataset,
                          batch_size=batch_size,
                          num_workers=2,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,
                          num_workers=2,
                          shuffle=True)

In [6]:
def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.xavier_uniform_(m.weight.data)
#         torch.nn.init.xavier_uniform(m.bias.data)
        m.bias.data.zero_()
epochs     = 100
start_epoch = 0
#criterion = torch.nn.MSELoss()
criterion = torch.nn.CrossEntropyLoss()
fcn_model = FCN_bak(n_class=34)
# fcn_model.apply(init_weights)
fcn_model.load_state_dict(torch.load('best_model_02_11_03_50.pt'))
optimizer = optim.Adam(fcn_model.parameters(), lr=5e-6, weight_decay=1e-2)

In [7]:
dt = datetime.now().strftime("%m_%d_%H_%M")
output_fn = "aug_model_output_" + dt + ".txt"
best_model_fn = "aug_best_model_" + dt + ".pt"
model_fn = "aug_model_" + dt + ".pt"

def print_info(out_str):
    f = open(output_fn,"a")
    print(out_str)
    f.write(out_str)
    f.close()

print_info("Started: %s\nFrom a previously trained model which left off on start of epoch 9.\n" % datetime.now())

Started: 2020-02-15 17:45:54.831319
From a previously trained model which left off on start of epoch 9.



In [ ]:
use_gpu = torch.cuda.is_available()
# use_gpu = False
if use_gpu:
    fcn_model = fcn_model.to(device)
    
best_loss = float('inf')
prev_loss = float('inf')
loss_inc_cnt = 0
stop_early = False
report_classes = {'building': 0, 'traffic sign': 0, 'person': 0, 'car': 0, 'bicycle': 0 }
rprt_building = 0
rprt_traffic = 0
rprt_person = 0
rprt_car = 0
rprt_bicycle = 0
building = 2
traffic_sign = 7
person = 11
car = 13
bicycle = 18

def train():
    softmax = nn.Softmax(dim=1)
    print("Starting Training")

    for epoch in range(start_epoch, epochs):
        
        ts = time.time()
        for iter, (X, tar, Y) in enumerate(train_loader):
            optimizer.zero_grad()

            if use_gpu:
                inputs = X.to(device)
                labels_cat = Y.to(device)
            else:
                inputs, labels_cat, labels_enc = X, Y, tar

            outputs = softmax(fcn_model(inputs))
            loss = criterion(outputs, labels_cat)
            loss.backward()
            optimizer.step()

            if iter % 10 == 0:
                print_info("epoch{}, iter{}, loss: {} \n".format(epoch, iter, loss.item()))
                
        
        print_info("Finish epoch {}, time elapsed {} \n".format(epoch, time.time() - ts))
    
        loss, acc, IoU, rprt_building, rprt_traffic, rprt_person, rprt_car, rprt_bicycle = evaluate(train_loader)

        print_info("Training Check:\tLoss: %f\tAccuracy: %f\tIoU: %f \n" % (loss, acc * 100, IoU))
        
        val(epoch)
        if stop_early: return
  
#         fcn_model.train()

def evaluate(data_loader, validation=False, verbose=True):

    global best_loss
    global prev_loss
    global loss_inc_cnt
    global stop_early
    global report_classes
    
    with torch.no_grad():
        losses = []
        accs = []
        ious = []
        rprt = {'building': [], 'traffic sign': [], 'person': [], 'car': [], 'bicycle': [] }
        softmax = nn.Softmax(dim=1)
        ts = time.time()
        print("Starting Evaluation")
        
        for iter, (X, tar, Y) in enumerate(data_loader):

            if use_gpu:
                inputs = X.to(device)
                labels_cat = Y.to(device)
            else:
                inputs, labels_cat, labels_enc = X, Y, tar

            outputs = fcn_model(inputs)
            outputs = softmax(outputs)

            output_labels = outputs.argmax(dim=1)

            losses.append(criterion(outputs, labels_cat).item())

            accs.append(pixel_acc(output_labels, labels_cat))
            
            iou_res = iou(output_labels, labels_cat)
            rprt['building'].append(iou_res[building])
            rprt['traffic sign'].append(iou_res[traffic_sign])
            rprt['person'].append(iou_res[person])
            rprt['car'].append(iou_res[car])
            rprt['bicycle'].append(iou_res[bicycle])
            ious.append(np.nanmean(iou_res))

            loss = np.mean(losses)
            acc = np.mean(accs)
            IoU = np.mean(ious)
            
            
            if verbose: print("Batch %d:\tLoss: %f\tAccuracy: %f\tIoU: %f" % (iter, loss, acc * 100, IoU))
            

        print("Finished evaluation. Time elapsed %f" % (time.time() - ts))

        # This probably should not be a straight average, but just doing this for now
        loss = np.mean(losses)
        acc = np.mean(accs)
        IoU = np.mean(ious)
        rprt_building = np.mean(rprt['building'])
        rprt_traffic = np.mean(rprt['traffic sign'])
        rprt_person = np.mean(rprt['person'])
        rprt_car = np.mean(rprt['car'])
        rprt_bicycle = np.mean(rprt['bicycle'])       
        
        if validation:
            if best_loss > loss:
                best_loss = loss
                print_info("Best Loss: " + str(best_loss) + "\n")
                torch.save(fcn_model.state_dict(), best_model_fn)
            loss_inc_cnt = loss_inc_cnt + 1 if prev_loss < loss else 0
            if loss_inc_cnt > 3: stop_early = True
            torch.save(fcn_model.state_dict(), model_fn)
        
        return loss, acc, IoU, rprt_building, rprt_traffic, rprt_person, rprt_car, rprt_bicycle

def val(epoch):
    # fcn_model.eval()
    # Complete this function - Calculate loss, accuracy and IoU for every epoch
    # Make sure to include a softmax after the output from your model
    loss, acc, IoU, rprt_building, rprt_traffic, rprt_person, rprt_car, rprt_bicycle = evaluate(val_loader, validation=True)
    print_info("Validation Results: Loss: %f\tAccuracy: %f\tIoU: %f \n" % (loss, acc * 100, IoU))
    if stop_early: print_info("Epoch %d:\tStopping Early" % (epoch))
    
def test():
    print(' ')
    # Complete this function - Calculate accuracy and IoU 
    # Make sure to include a softmax after the output from your model
    loss, acc, IoU, rprt_building, rprt_traffic, rprt_person, rprt_car, rprt_bicycle = evaluate(test_loader)
    print_info("Test Results:\tLoss: %f\tAccuracy: %f\tIoU: %f \n" % (loss, acc * 100, IoU))
    
if __name__ == "__main__":
#     val(0)  # show the accuracy before training
#     print_info("---------Above is accuracy before training.---------\n")
    train()
    test()

Starting Training
epoch0, iter0, loss: 2.979325771331787 

epoch0, iter10, loss: 2.9778151512145996 

epoch0, iter20, loss: 2.9655869007110596 

Finish epoch 0, time elapsed 74.54431104660034 

Starting Evaluation
Batch 0:	Loss: 3.009636	Accuracy: 56.639576	IoU: 0.232412
Batch 1:	Loss: 3.005926	Accuracy: 57.097483	IoU: 0.217927
Batch 2:	Loss: 2.977707	Accuracy: 59.904569	IoU: 0.225319
Batch 3:	Loss: 2.979242	Accuracy: 59.779320	IoU: 0.227405
Batch 4:	Loss: 2.995774	Accuracy: 58.104584	IoU: 0.220264
Batch 5:	Loss: 2.989628	Accuracy: 58.728766	IoU: 0.220083
Batch 6:	Loss: 2.994937	Accuracy: 58.222419	IoU: 0.215695
Batch 7:	Loss: 2.990748	Accuracy: 58.629391	IoU: 0.218206
Batch 8:	Loss: 2.987776	Accuracy: 58.915732	IoU: 0.221373
Batch 9:	Loss: 2.991779	Accuracy: 58.506846	IoU: 0.218284
Batch 10:	Loss: 2.985370	Accuracy: 59.151796	IoU: 0.222159
Batch 11:	Loss: 2.980140	Accuracy: 59.668729	IoU: 0.223365
Batch 12:	Loss: 2.978432	Accuracy: 59.836544	IoU: 0.222695
Batch 13:	Loss: 2.976300	Accu

Batch 21:	Loss: 2.963460	Accuracy: 61.306784	IoU: 0.229367
Batch 22:	Loss: 2.962126	Accuracy: 61.438841	IoU: 0.229117
Batch 23:	Loss: 2.961499	Accuracy: 61.502924	IoU: 0.229619
Batch 24:	Loss: 2.962090	Accuracy: 61.442501	IoU: 0.229185
Batch 25:	Loss: 2.963489	Accuracy: 61.302103	IoU: 0.228077
Batch 26:	Loss: 2.963542	Accuracy: 61.296888	IoU: 0.227925
Batch 27:	Loss: 2.962631	Accuracy: 61.387726	IoU: 0.227572
Batch 28:	Loss: 2.961071	Accuracy: 61.542177	IoU: 0.228566
Batch 29:	Loss: 2.962419	Accuracy: 61.405927	IoU: 0.227886
Finished evaluation. Time elapsed 80.689750
Training Check:	Loss: 2.962419	Accuracy: 61.405927	IoU: 0.227886 

Starting Evaluation
Batch 0:	Loss: 2.997202	Accuracy: 57.941122	IoU: 0.216605
Batch 1:	Loss: 3.008548	Accuracy: 56.779695	IoU: 0.203973
Batch 2:	Loss: 3.019959	Accuracy: 55.626335	IoU: 0.199156
Batch 3:	Loss: 3.003106	Accuracy: 57.319398	IoU: 0.208643
Batch 4:	Loss: 2.998274	Accuracy: 57.798225	IoU: 0.213262
Batch 5:	Loss: 2.994554	Accuracy: 58.175635	IoU:

Batch 13:	Loss: 2.996286	Accuracy: 57.989327	IoU: 0.207464
Batch 14:	Loss: 2.996350	Accuracy: 57.990147	IoU: 0.208274
Batch 15:	Loss: 2.996025	Accuracy: 58.021094	IoU: 0.208360
Batch 16:	Loss: 2.995048	Accuracy: 58.117220	IoU: 0.208080
Batch 17:	Loss: 2.993817	Accuracy: 58.245346	IoU: 0.209160
Batch 18:	Loss: 2.995154	Accuracy: 58.112488	IoU: 0.208867
Batch 19:	Loss: 2.995248	Accuracy: 58.103680	IoU: 0.208724
Finished evaluation. Time elapsed 54.585288
Validation Results: Loss: 2.995248	Accuracy: 58.103680	IoU: 0.208724 

epoch5, iter0, loss: 2.938392162322998 

epoch5, iter10, loss: 2.9953150749206543 

epoch5, iter20, loss: 2.9558205604553223 

Finish epoch 5, time elapsed 74.7019567489624 

Starting Evaluation
Batch 0:	Loss: 2.957098	Accuracy: 61.879644	IoU: 0.224577
Batch 1:	Loss: 2.961967	Accuracy: 61.419673	IoU: 0.223104
Batch 2:	Loss: 2.968698	Accuracy: 60.741828	IoU: 0.215318
Batch 3:	Loss: 2.957839	Accuracy: 61.854722	IoU: 0.225456
Batch 4:	Loss: 2.970701	Accuracy: 60.561449	I

Batch 13:	Loss: 2.959936	Accuracy: 61.673153	IoU: 0.226645
Batch 14:	Loss: 2.959749	Accuracy: 61.690534	IoU: 0.225207
Batch 15:	Loss: 2.959565	Accuracy: 61.706712	IoU: 0.224440
Batch 16:	Loss: 2.960846	Accuracy: 61.574959	IoU: 0.224395
Batch 17:	Loss: 2.964008	Accuracy: 61.261575	IoU: 0.224368
Batch 18:	Loss: 2.966070	Accuracy: 61.058571	IoU: 0.223010
Batch 19:	Loss: 2.967178	Accuracy: 60.949760	IoU: 0.222562
Batch 20:	Loss: 2.970952	Accuracy: 60.570323	IoU: 0.221396
Batch 21:	Loss: 2.972184	Accuracy: 60.447822	IoU: 0.221006
Batch 22:	Loss: 2.971844	Accuracy: 60.481840	IoU: 0.221502
Batch 23:	Loss: 2.970669	Accuracy: 60.596399	IoU: 0.222626
Batch 24:	Loss: 2.969139	Accuracy: 60.747226	IoU: 0.222920
Batch 25:	Loss: 2.967524	Accuracy: 60.907347	IoU: 0.224194
Batch 26:	Loss: 2.967721	Accuracy: 60.885411	IoU: 0.224808
Batch 27:	Loss: 2.969681	Accuracy: 60.688151	IoU: 0.225185
Batch 28:	Loss: 2.967915	Accuracy: 60.864574	IoU: 0.225686
Batch 29:	Loss: 2.967677	Accuracy: 60.888584	IoU: 0.2251

Batch 6:	Loss: 2.992040	Accuracy: 58.420237	IoU: 0.209649
Batch 7:	Loss: 2.991964	Accuracy: 58.425345	IoU: 0.207757
Batch 8:	Loss: 2.996655	Accuracy: 57.955017	IoU: 0.206495
Batch 9:	Loss: 2.993447	Accuracy: 58.281061	IoU: 0.206705
Batch 10:	Loss: 2.992224	Accuracy: 58.400110	IoU: 0.207050
Batch 11:	Loss: 2.990358	Accuracy: 58.585501	IoU: 0.208970
Batch 12:	Loss: 2.991912	Accuracy: 58.428723	IoU: 0.208634
Batch 13:	Loss: 2.991712	Accuracy: 58.452471	IoU: 0.208536
Batch 14:	Loss: 2.990911	Accuracy: 58.533193	IoU: 0.208472
Batch 15:	Loss: 2.988326	Accuracy: 58.792855	IoU: 0.209573
Batch 16:	Loss: 2.989940	Accuracy: 58.629463	IoU: 0.208867
Batch 17:	Loss: 2.991773	Accuracy: 58.444578	IoU: 0.208247
Batch 18:	Loss: 2.993870	Accuracy: 58.233964	IoU: 0.208322
Batch 19:	Loss: 2.992759	Accuracy: 58.343435	IoU: 0.209816
Finished evaluation. Time elapsed 60.345180
Best Loss: 2.992758810520172

Validation Results: Loss: 2.992759	Accuracy: 58.343435	IoU: 0.209816 

epoch10, iter0, loss: 2.984177112

...

Batch 5:	Loss: 2.992992	Accuracy: 58.322185	IoU: 0.206305
Batch 6:	Loss: 2.991374	Accuracy: 58.482413	IoU: 0.207357
Batch 7:	Loss: 2.989351	Accuracy: 58.683627	IoU: 0.207892
Batch 8:	Loss: 2.993715	Accuracy: 58.243761	IoU: 0.207526
Batch 9:	Loss: 2.991809	Accuracy: 58.438532	IoU: 0.208535
Batch 10:	Loss: 2.991964	Accuracy: 58.426172	IoU: 0.208609
Batch 11:	Loss: 2.996647	Accuracy: 57.955225	IoU: 0.207674
Batch 12:	Loss: 2.996364	Accuracy: 57.982393	IoU: 0.207167
Batch 13:	Loss: 2.996218	Accuracy: 58.000770	IoU: 0.207493
Batch 14:	Loss: 2.996615	Accuracy: 57.959410	IoU: 0.208372
Batch 15:	Loss: 2.996307	Accuracy: 57.990677	IoU: 0.207840
Batch 16:	Loss: 2.995716	Accuracy: 58.049931	IoU: 0.207546
Batch 17:	Loss: 2.993480	Accuracy: 58.273781	IoU: 0.207774
Batch 18:	Loss: 2.994245	Accuracy: 58.196660	IoU: 0.207709
Batch 19:	Loss: 2.995398	Accuracy: 58.080326	IoU: 0.208411
Finished evaluation. Time elapsed 56.512235
Validation Results: Loss: 2.995398	Accuracy: 58.080326	IoU: 0.208411 

epoch

Batch 5:	Loss: 2.951031	Accuracy: 62.515481	IoU: 0.229146
Batch 6:	Loss: 2.952830	Accuracy: 62.344735	IoU: 0.227105
Batch 7:	Loss: 2.949721	Accuracy: 62.660079	IoU: 0.229261
Batch 8:	Loss: 2.946547	Accuracy: 62.980642	IoU: 0.229438
Batch 9:	Loss: 2.942736	Accuracy: 63.359400	IoU: 0.230550
Batch 10:	Loss: 2.946777	Accuracy: 62.952407	IoU: 0.229165
Batch 11:	Loss: 2.943316	Accuracy: 63.299689	IoU: 0.230951
Batch 12:	Loss: 2.944894	Accuracy: 63.140578	IoU: 0.230817
Batch 13:	Loss: 2.946656	Accuracy: 62.965236	IoU: 0.229918
Batch 14:	Loss: 2.948727	Accuracy: 62.756433	IoU: 0.228088
Batch 15:	Loss: 2.947324	Accuracy: 62.897516	IoU: 0.229250
Batch 16:	Loss: 2.947005	Accuracy: 62.927470	IoU: 0.229991
Batch 17:	Loss: 2.946938	Accuracy: 62.935477	IoU: 0.229955
Batch 18:	Loss: 2.947766	Accuracy: 62.853343	IoU: 0.229272
Batch 19:	Loss: 2.948227	Accuracy: 62.805933	IoU: 0.229098
Batch 20:	Loss: 2.951708	Accuracy: 62.457827	IoU: 0.228911
Batch 21:	Loss: 2.952788	Accuracy: 62.352912	IoU: 0.229116
Ba

Training Check:	Loss: 2.966774	Accuracy: 60.969543	IoU: 0.226035 

Starting Evaluation
Batch 0:	Loss: 3.027509	Accuracy: 54.858522	IoU: 0.185745
Batch 1:	Loss: 3.015809	Accuracy: 56.025043	IoU: 0.188471
Batch 2:	Loss: 3.002093	Accuracy: 57.392101	IoU: 0.202430
Batch 3:	Loss: 3.012207	Accuracy: 56.380355	IoU: 0.200921
Batch 4:	Loss: 3.002812	Accuracy: 57.322008	IoU: 0.204980
Batch 5:	Loss: 3.001531	Accuracy: 57.447677	IoU: 0.203615
Batch 6:	Loss: 3.002568	Accuracy: 57.354171	IoU: 0.202810
Batch 7:	Loss: 2.996954	Accuracy: 57.915354	IoU: 0.204449
Batch 8:	Loss: 2.989886	Accuracy: 58.623392	IoU: 0.206026
Batch 9:	Loss: 2.987614	Accuracy: 58.852723	IoU: 0.209008
Batch 10:	Loss: 2.990593	Accuracy: 58.552224	IoU: 0.207948
Batch 11:	Loss: 2.988988	Accuracy: 58.714264	IoU: 0.207779
Batch 12:	Loss: 2.989919	Accuracy: 58.620468	IoU: 0.207873
Batch 13:	Loss: 2.991073	Accuracy: 58.507647	IoU: 0.207211
Batch 14:	Loss: 2.993835	Accuracy: 58.230788	IoU: 0.206803
Batch 15:	Loss: 2.997552	Accuracy: 57.

epoch56, iter20, loss: 2.9978766441345215 

Finish epoch 56, time elapsed 75.88591480255127 

Starting Evaluation
Batch 0:	Loss: 2.970345	Accuracy: 60.676022	IoU: 0.243583
Batch 1:	Loss: 2.951508	Accuracy: 62.529917	IoU: 0.234788
Batch 2:	Loss: 2.949437	Accuracy: 62.721694	IoU: 0.227856
Batch 3:	Loss: 2.951696	Accuracy: 62.486937	IoU: 0.228715
Batch 4:	Loss: 2.955528	Accuracy: 62.097469	IoU: 0.229282
Batch 5:	Loss: 2.951446	Accuracy: 62.499620	IoU: 0.231520
Batch 6:	Loss: 2.948135	Accuracy: 62.831539	IoU: 0.233116
Batch 7:	Loss: 2.946332	Accuracy: 63.004746	IoU: 0.236260
Batch 8:	Loss: 2.947289	Accuracy: 62.917111	IoU: 0.236073
Batch 9:	Loss: 2.950644	Accuracy: 62.588416	IoU: 0.234764
Batch 10:	Loss: 2.954430	Accuracy: 62.207968	IoU: 0.233099
Batch 11:	Loss: 2.956561	Accuracy: 61.997985	IoU: 0.232087
Batch 12:	Loss: 2.952994	Accuracy: 62.352604	IoU: 0.233447
Batch 13:	Loss: 2.956133	Accuracy: 62.034205	IoU: 0.232636
Batch 14:	Loss: 2.956353	Accuracy: 62.012392	IoU: 0.232040
Batch 15:	L

Batch 23:	Loss: 2.961940	Accuracy: 61.444308	IoU: 0.226192
Batch 24:	Loss: 2.965346	Accuracy: 61.103729	IoU: 0.225081
Batch 25:	Loss: 2.965233	Accuracy: 61.116209	IoU: 0.225258
Batch 26:	Loss: 2.966999	Accuracy: 60.940174	IoU: 0.225523
Batch 27:	Loss: 2.966751	Accuracy: 60.964847	IoU: 0.226098
Batch 28:	Loss: 2.966192	Accuracy: 61.023287	IoU: 0.225901
Batch 29:	Loss: 2.966120	Accuracy: 61.035630	IoU: 0.225744
Finished evaluation. Time elapsed 80.782424
Training Check:	Loss: 2.966120	Accuracy: 61.035630	IoU: 0.225744 

Starting Evaluation
Batch 0:	Loss: 3.002595	Accuracy: 57.417183	IoU: 0.209394
Batch 1:	Loss: 2.976091	Accuracy: 60.038590	IoU: 0.214346
Batch 2:	Loss: 2.981844	Accuracy: 59.483852	IoU: 0.212450
Batch 3:	Loss: 2.998626	Accuracy: 57.788079	IoU: 0.203909
Batch 4:	Loss: 2.989045	Accuracy: 58.743528	IoU: 0.211839
Batch 5:	Loss: 2.991610	Accuracy: 58.491546	IoU: 0.211510
Batch 6:	Loss: 2.994074	Accuracy: 58.237487	IoU: 0.209946
Batch 7:	Loss: 2.993056	Accuracy: 58.333213	IoU: 0